In [28]:
#API: I37T0DG04XQQ79GE
#reference: https://www.alphavantage.co/documentation/

❚ Required: function

The function of your choice. In this case, function=NEWS_SENTIMENT

❚ Optional: tickers

The stock/crypto/forex symbols of your choice. For example: tickers=IBM will filter for articles that mention the IBM ticker; tickers=COIN,CRYPTO:BTC,FOREX:USD will filter for articles that simultaneously mention Coinbase (COIN), Bitcoin (CRYPTO:BTC), and US Dollar (FOREX:USD) in their content.

❚ Optional: topics

The news topics of your choice. For example: topics=technology will filter for articles that write about the technology sector; topics=technology,ipo will filter for articles that simultaneously cover technology and IPO in their content. Below is the full list of supported topics:

Blockchain: blockchain
Earnings: earnings
IPO: ipo
Mergers & Acquisitions: mergers_and_acquisitions
Financial Markets: financial_markets
Economy - Fiscal Policy (e.g., tax reform, government spending): economy_fiscal
Economy - Monetary Policy (e.g., interest rates, inflation): economy_monetary
Economy - Macro/Overall: economy_macro
Energy & Transportation: energy_transportation
Finance: finance
Life Sciences: life_sciences
Manufacturing: manufacturing
Real Estate & Construction: real_estate
Retail & Wholesale: retail_wholesale
Technology: technology


❚ Optional: time_from and time_to

The time range of the news articles you are targeting, in YYYYMMDDTHHMM format. For example: time_from=20220410T0130. If time_from is specified but time_to is missing, the API will return articles published between the time_from value and the current time.

❚ Optional: sort

By default, sort=LATEST and the API will return the latest articles first. You can also set sort=EARLIEST or sort=RELEVANCE based on your use case.

❚ Optional: limit

By default, limit=50 and the API will return up to 50 matching results. You can also set limit=200 to output up to 200 results. If you are looking for an even higher output limit, please contact support@alphavantage.co to have your limit boosted.

❚ Required: apikey

Your API key. Claim your free API key here.

In [95]:
import requests
import pandas as pd
import numpy as np
class cryptoNews:
    
    def __init__(self, tickers ='AAPL', topics = '', \
                 time_from = '20220918T0000', time_to = '', sort='EARLIEST', limit=50, apikey = 'I37T0DG04XQQ79GE'):
        
        self.function = 'NEWS_SENTIMENT'
        self.tickers = tickers
        self.topics = topics
        self.time_from = time_from
        self.time_to = time_to
        self.sort = sort
        self.limit = limit
        self.apikey = apikey
        self.url = """https://www.alphavantage.co/query?function={}&tickers={}&topics={}&time_from={}&time_to={}&sort={}&limit={}&apikey={}""".format(self.function, self.tickers, self.topics, \
                                                                                                                                                      self.time_from, self.time_to, self.sort, \
                                                                                                                                                      self.limit, self.apikey) 
        self.DataFrame = None
        
    def getJson(self):
        
        r = requests.get(self.url)
        data = r.json()
        
        return data
    
    def getfeed(self):
        return self.getJson()['feed']
    
    def getDataFrame(self):
        json = self.getJson()
        items = json['items']
        feed = json['feed']
        self.DataFrame = pd.DataFrame.from_dict(feed)
        return self.DataFrame

# Demo
Refer to https://www.alphavantage.co/documentation/
- Don't need to change topics 
- time_from format: YYYYMMDDTHHHMM Example: "20220410T0130" means 2022-04-10, 1:30am UTC.
- time_to format: the same as time_to if none, using the current time
- sort: order: "EARLIEST", "LAST", "RELEVANCE" "EARLIEST" by default
- limit: number of articles
- apikey: the author of this notebook already got the apikey. You can use his but feel free to get your own one

*Note*: The limit of each query should be 200 so if you want to get long historical data, you can create cryptoNews iteratively by segmenting long history into many small time periods (1 or 2 days?)

Say we want to get all news about crypto btc from 2022-09-18, 00:00am UTC to now.

In [96]:
newsdata = cryptoNews(tickers = 'CRYPTO:BTC', time_from = '20220918T0000')

In [99]:
newsdata.getJson()

{'items': '18',
 'sentiment_score_definition': 'x <= -0.35: Bearish; -0.35 < x <= -0.15: Somewhat-Bearish; -0.15 < x < 0.15: Neutral; 0.15 <= x < 0.35: Somewhat_Bullish; x >= 0.35: Bullish',
 'relevance_score_definition': '0 < x <= 1, with a higher score indicating higher relevance.',
 'feed': [{'title': 'Cryptocurrency Prices Today September 18: Major cryptos flatline as volumes decline',
   'url': 'https://www.moneycontrol.com/news/business/cryptocurrency/cryptocurrency-prices-today-september-18-major-cryptos-flatline-as-volumes-decline-9193971.html',
   'time_published': '20220918T030751',
   'authors': [],
   'summary': 'The total volume in DeFi stood at $3.86 billion, 8.02 percent of the total crypto market 24-hour volume.',
   'banner_image': 'https://images.moneycontrol.com/static-mcnews/2022/09/Cryptocurrency-1-770x433.png',
   'source': 'Money Control',
   'category_within_source': 'Business',
   'source_domain': 'www.moneycontrol.com',
   'topics': [{'topic': 'Financial Marke

In [100]:
newsdata.getfeed()

[{'title': 'Cryptocurrency Prices Today September 18: Major cryptos flatline as volumes decline',
  'url': 'https://www.moneycontrol.com/news/business/cryptocurrency/cryptocurrency-prices-today-september-18-major-cryptos-flatline-as-volumes-decline-9193971.html',
  'time_published': '20220918T030751',
  'authors': [],
  'summary': 'The total volume in DeFi stood at $3.86 billion, 8.02 percent of the total crypto market 24-hour volume.',
  'banner_image': 'https://images.moneycontrol.com/static-mcnews/2022/09/Cryptocurrency-1-770x433.png',
  'source': 'Money Control',
  'category_within_source': 'Business',
  'source_domain': 'www.moneycontrol.com',
  'topics': [{'topic': 'Financial Markets', 'relevance_score': '0.158519'},
   {'topic': 'Blockchain', 'relevance_score': '0.576289'}],
  'overall_sentiment_score': 0.095389,
  'overall_sentiment_label': 'Neutral',
  'ticker_sentiment': [{'ticker': 'CRYPTO:BTC',
    'relevance_score': '0.238133',
    'ticker_sentiment_score': '0.10637',
    

In [102]:
newsdata.getDataFrame().head()

,title,url,time_published,authors,summary,banner_image,source,category_within_source,source_domain,topics,overall_sentiment_score,overall_sentiment_label,ticker_sentiment
0,Cryptocurrency Prices Today September 18: Majo...,https://www.moneycontrol.com/news/business/cry...,20220918T030751,[],The total volume in DeFi stood at $3.86 billio...,https://images.moneycontrol.com/static-mcnews/...,Money Control,Business,www.moneycontrol.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.095389,Neutral,"[{'ticker': 'CRYPTO:BTC', 'relevance_score': '..."
1,What are the disadvantages of blockchain | The...,https://www.financialexpress.com/blockchain/wh...,20220918T060410,[],What are the disadvantages of blockchain The F...,https://www.financialexpress.com/wp-content/up...,The Financial Express,IndustryGoogleRSS,www.financialexpress.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.043527,Neutral,"[{'ticker': 'CRYPTO:ETH', 'relevance_score': '..."
2,Binance misallocates $20M Helium's HNT tokens ...,https://www.financialexpress.com/blockchain/bi...,20220918T075821,[],Binance misallocates $20M Helium's HNT tokens ...,https://www.financialexpress.com/wp-content/up...,The Financial Express,MarketsGoogleRSS,www.financialexpress.com,[],-0.055256,Neutral,"[{'ticker': 'CRYPTO:BTC', 'relevance_score': '..."
3,South Korea recommends enactment of metaverse ...,https://www.financialexpress.com/blockchain/so...,20220918T085336,[],South Korea recommends enactment of metaverse ...,https://www.financialexpress.com/wp-content/up...,The Financial Express,IndustryGoogleRSS,www.financialexpress.com,[],0.108967,Neutral,"[{'ticker': 'CRYPTO:BTC', 'relevance_score': '..."
4,"Vietnam leads global crypto adoption, US comes...",https://www.financialexpress.com/blockchain/vi...,20220918T091455,[],"Vietnam leads global crypto adoption, US comes...",https://www.financialexpress.com/wp-content/up...,The Financial Express,MarketsGoogleRSS,www.financialexpress.com,"[{'topic': 'Blockchain', 'relevance_score': '0...",0.212005,Somewhat-Bullish,"[{'ticker': 'CRYPTO:BTC', 'relevance_score': '..."


In [ ]:
|